2018314848 한승희
기계학습 프로젝트 발표

In [0]:
import numpy as np
import pandas as pd

1. 분류기 선정: Naive Bayes? SVM? Decision Tree? LR?

직접 모든부분을 구현하는 실습을 한건 Naive Bayes & Decision Tree

그 중 Post로 되어있는 데이터로 자살 위험도를 측정하기에 가장 좋은 모델은?

->NB일것이다!

In [47]:
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import names
all_names = set(names.words())

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [49]:
Train_csv = pd.read_csv('/content/drive/My Drive/TRAIN.csv')
Train_csv.head()

,Unnamed: 0,User,Post,Label,LabelNumber
0,5,user-5,"['Since I dont know what DBT is, would you min...",Supportive,4.0
1,116,user-116,['I didnt... I always feel as though Im a fail...,Ideation,2.0
2,45,user-45,['Thanks for this. I was working with my dad t...,Ideation,2.0
3,16,user-16,"['oh, doh.I had someone tell me similar recent...",Ideation,2.0
4,462,user-462,['I know the feeling of not wanting to be a bu...,Ideation,2.0


In [50]:
Train_csv.drop('User', axis=1, inplace=True)
Train_csv.drop('Unnamed: 0', axis=1, inplace=True)
Train_csv.drop('Label', axis=1, inplace=True)

Train_csv.head()

,Post,LabelNumber
0,"['Since I dont know what DBT is, would you min...",4.0
1,['I didnt... I always feel as though Im a fail...,2.0
2,['Thanks for this. I was working with my dad t...,2.0
3,"['oh, doh.I had someone tell me similar recent...",2.0
4,['I know the feeling of not wanting to be a bu...,2.0


In [0]:
def clean_text(doc):
  cleaned_post = []
  for word in doc.split():
    word = word.lower()
    if word.isalpha() and word not in all_names and len(word) > 2:
      cleaned_post.append(lemmatizer.lemmatize(word))
  return ' '.join(cleaned_post)

In [52]:
clean_posts = []
for i in range(len(Train_csv.Post)):
  clean_posts.append(clean_text(Train_csv.Post[i]))
x = pd.Series(clean_posts)
Train_csv['Post'] = x
Train_csv

,Post,LabelNumber
0,dont know what dbt would you mind explaining l...,4.0
1,always feel though failure when dont when say ...,2.0
2,for wa working with dad today grandma house an...,2.0
3,had someone tell similar took calling out some...,2.0
4,know the feeling not wanting burden your had r...,2.0
...,...,...
345,sure you hear this all the time but where you ...,2.0
346,know what you that proverbial downward spiral ...,3.0
347,glad you started the fight started you are the...,2.0
348,just happened browse through this and saw your...,3.0


In [53]:
Post_1 = Train_csv.Post.values
print(type(Post_1))
"""
문서1 글들
문서2 글들
문서3 글들
4
5
6
7
8
9
[글, 글, 글, 글, 글, 글]
"""
Post_1.shape, len(Post_1[0]), type(Post_1[0])

<class 'numpy.ndarray'>


((350,), 7805, str)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# cv = CountVectorizer(stop_words = 'english', max_features=2500)
# # cv = TfidfVectorizer(max_df=0.5, stop_words = 'english', max_features = 5000)
# term_docs = cv.fit_transform(clean_posts)
# term_docs.shape

In [0]:
def get_label_index(Dataframe):
  label_index = {0:[], 1:[], 2:[], 3:[], 4:[]}
  posts = Dataframe.Post.values
  LabelNumbers = Dataframe.LabelNumber.values
  for idx, label in enumerate(LabelNumbers):
    label_index[label].append(idx)
  return label_index
# label_index = get_label_index(Train_csv)
# print(label_index)

In [0]:
def get_prior(label_index):
  prior = {label: len(index) for label, index in label_index.items()}
  total_sum = sum(prior.values())
  for i in prior.keys():
    prior[i] /= total_sum
  return prior
# prior = get_prior(label_index)
# prior

In [57]:
%%time
def get_likelihood(term_document_matrix, label_index, smoothing=0):
  likelihood = {}
  for label, index in label_index.items():
    likelihood[label] = term_document_matrix[index, :].sum(axis=0) + smoothing
    likelihood[label] = np.asarray(likelihood[label])[0]
    total_count = likelihood[label].sum()
    likelihood[label] = likelihood[label] / total_count
  return likelihood

# smoothing = 1
# likelihood = get_likelihood(term_docs, label_index, smoothing)
# print("우도값 shape : {}\n우도값 array :\n{}".format(
#     likelihood[0].shape, # 0번 레이블일 때 단어별 우도값 계산
#     likelihood[0][:20])) # 0번 레이블의 단어별 우도값 샘플 [:20]

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


In [0]:
def get_posterior(term_document_matrix, prior, likelihood):
  num_docs, posteriors = term_document_matrix.shape[0], []
  for i in range(num_docs):
    posterior = {key: np.log(prior_label) for key, prior_label in prior.items()}
    for label, likelihood_label in likelihood.items():
      term_document_vector = term_document_matrix.getrow(i)
      counts  = term_document_vector.data
      indices = term_document_vector.indices
      for count, index in zip(counts, indices):
        posterior[label] += np.log(likelihood_label[index]) * count
    min_log_posterior = min(posterior.values())
    for label in posterior:
      try:    posterior[label] = np.exp(posterior[label] - min_log_posterior)
      except: posterior[label] = float('inf')
    sum_posterior = sum(posterior.values())
    for label in posterior:
      if posterior[label] == float('inf'): posterior[label] = 1.0
      else: posterior[label] /= sum_posterior
    posteriors.append(posterior.copy())
  return posteriors
# posteriors = get_posterior(term_docs, prior, likelihood)
# print(len(posteriors),posteriors)

In [0]:
from sklearn.model_selection import StratifiedKFold
k = 10
k_fold = StratifiedKFold(n_splits=k)

clean_posts_np = np.array(clean_posts)
labels_np = Train_csv.LabelNumber.values

max_features_option = [2000, 3000, 4000, 5000]
smoothing_option = [0.5, 1.0, 1.5, 2.0]
cv_option = ['TfidfVectorizer', 'CountVectorizer']


In [60]:
%%time
from sklearn.metrics import classification_report
report = None
max_accuracy = 0
max_max_feature = max_smoothing = max_vectorizer = None
for train_indices, test_indices in k_fold.split(clean_posts_np, labels_np):
  X_train, X_test = clean_posts_np[train_indices], clean_posts_np[test_indices]
  Y_train, Y_test = labels_np[train_indices], labels_np[test_indices]
  
  for max_feature in max_features_option:
    for vectorizer in cv_option:
      if vectorizer == 'TfidfVectorizer':
        cv = TfidfVectorizer(stop_words="english", max_features=max_feature)
      else:
        cv = CountVectorizer(stop_words="english", max_features=max_feature)
      # cv = TfidfVectorizer(stop_words="english", max_features=max_feature)

      term_docs_train = cv.fit_transform(X_train)
      term_docs_test = cv.transform(X_test)

      for smoothing in smoothing_option:
        accuracy, y_pred = my_machine_learning(smoothing)
        if accuracy > max_accuracy:
          max_accuracy = accuracy
          max_max_feature = max_feature
          max_smoothing = smoothing
          max_vectorizer = vectorizer
          report = classification_report(Y_test, y_pred)
print(f"\n\n제 모델은 {max_vectorizer}로 벡터화하고 max_features가 {max_max_feature}, 스무딩이 {max_smoothing}일때\n{max_accuracy*100:.1f}% 정확도로 가장 높습니다\n\n")
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classificat



제 모델은 CountVectorizer로 벡터화하고 max_features가 3000, 스무딩이 1.5일때
57.1% 정확도로 가장 높습니다


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       0.50      0.50      0.50         6
         2.0       0.53      0.69      0.60        13
         3.0       1.00      0.14      0.25         7
         4.0       0.64      1.00      0.78         7

    accuracy                           0.57        35
   macro avg       0.53      0.47      0.43        35
weighted avg       0.61      0.57      0.51        35

CPU times: user 48.2 s, sys: 47.4 ms, total: 48.2 s
Wall time: 48.6 s


In [0]:
from collections import defaultdict
def my_machine_learning(smoothing):

  label_index = defaultdict(list)
  for idx, label in enumerate(Y_train):
    label_index[label].append(idx)
  
  prior = get_prior(label_index)

  likelihood = get_likelihood(term_docs_train, label_index, smoothing)

  posterior = get_posterior(term_docs_test, prior, likelihood)

  correct = 0.0

  y_pred = []
  for i in posterior:
    its_label = -1
    label_prob = 0
    for key, value in i.items():
      if value > label_prob:
        its_label = key
        label_prob = value
    y_pred.append(its_label)

  for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
      correct += 1
  return correct/len(y_pred), y_pred

In [62]:
Test_csv = pd.read_csv('/content/drive/My Drive/TEST.csv')
Test_csv.head()

,Post
0,['Your dad loves you. Call him and tell him yo...
1,['Please believe what this person said. I can...
2,['I feel youre Ache pal. My world feels Hypera...
3,['I dont do much currently. I lifeguard for a ...
4,"['you always can kill yourself later:)', 'no n..."


In [63]:
clean_posts2 = []
for i in range(len(Test_csv.Post)):
  clean_posts2.append(clean_text(Test_csv.Post[i]))
x = pd.Series(clean_posts2)
Test_csv['Post'] = x
Test_csv

,Post
0,dad love call him and tell him youre having ha...
1,believe what this person cant stop cry the pot...
2,feel youre ache world feel hyperactive behavio...
3,dont much lifeguard for bit heading school tha...
4,always can kill yourself need try them just pi...
...,...
145,completely understand how you ive had moment h...
146,and get her seek will help
147,were still let tend help someone through diffi...
148,dont hyperactive behavior they just help take ...


In [64]:
Post_2 = Test_csv.Post.values
Post_2[:3]

array(['dad love call him and tell him youre having hard time and need some son did wouldnt feel the slightest bit that shit hyperactive behavior ninja paratrooper and wed fix that you need but need you think you get the fuck out that it killing your soul and that wound that tough went religious school whole wa basically told how what when could and this the time your life for you figure out who you not who someone else want you tell your parent you love and respect but you need what right for try telling them your gay first soften the then say not know how the same make reading comprehension much late there wa ocd diagnosis when wa wa you were too much it just part what make but dont let run think the book would have different use for when you read will help you understand the human how people hyperactive behavior spoken what tool you can use change how you are dont look video game strategy guide that will tell you exactly what it more hyperactive behavior user',
       'believe what 

In [65]:
Post_2.shape, len(Post_2[0]), type(Post_2[0])

((150,), 968, str)

K-fold를 바탕으로 파라미터를 작성해서 제출

-> 정확도 57프로가 아니라 30프로 Why?

-> Overfitting되었구나

나이브 베이즈에서 오버피팅시키는 요소가 무엇이 있을까?

In [66]:
# kfold를 바탕으로 파라미터 작성
cv = CountVectorizer(max_features=3800)
# cv = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words="english", max_features=3000)

term_docs_train = cv.fit_transform(clean_posts)
print(term_docs_train.shape)
term_docs_test = cv.transform(clean_posts2)
print(term_docs_test.shape)
label_index = get_label_index(Train_csv)

prior = get_prior(label_index)

smoothing = 1.0
likelihood = get_likelihood(term_docs_train, label_index, smoothing)

posterior = get_posterior(term_docs_test, prior, likelihood)

(350, 3800)
(150, 3800)


In [67]:
y_pred = []
for i in posterior:
  its_label = -1
  label_prob = 0
  for key, value in i.items():
    if value > label_prob:
      its_label = key
      label_prob = value
  y_pred.append(its_label)
len(y_pred)

150

In [68]:
y_pred = np.array(y_pred)
y_pred

array([4, 4, 2, 1, 2, 2, 4, 2, 3, 1, 2, 4, 1, 2, 4, 4, 4, 1, 4, 2, 1, 4,
       2, 4, 2, 4, 4, 4, 4, 2, 4, 1, 2, 2, 4, 2, 4, 1, 2, 4, 2, 3, 4, 2,
       2, 3, 4, 3, 1, 3, 4, 2, 2, 2, 3, 3, 1, 2, 3, 2, 2, 4, 2, 2, 3, 4,
       2, 2, 3, 2, 4, 1, 4, 4, 4, 4, 4, 3, 2, 4, 3, 2, 4, 4, 1, 2, 3, 4,
       4, 1, 4, 2, 3, 4, 2, 4, 4, 4, 2, 4, 4, 2, 2, 2, 2, 2, 3, 1, 2, 1,
       4, 2, 1, 1, 2, 2, 4, 1, 4, 2, 2, 1, 2, 2, 4, 2, 4, 2, 2, 1, 4, 4,
       4, 2, 4, 4, 1, 1, 2, 2, 1, 4, 2, 3, 4, 2, 2, 4, 1, 2])

30% -> 35% 상승!

In [69]:
pd.DataFrame(y_pred).to_csv("2018314848.csv", index = False)
x = pd.read_csv('2018314848.csv')
x.head()

,0
0,4
1,4
2,2
3,1
4,2
